In [1]:
from transformers import BertTokenizer
import numpy as np
from tqdm import tqdm
import tensorflow as tf

/media/mohamed/AC6030326030059C/CMP1Materials/Forth/First/NLP/Project/dataset/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-15 18:38:54.658026: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734280734.709472  125155 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734280734.725558  125155 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-15 18:38:54.849605: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to 

In [2]:
# processed_sentences_dev = np.load('processed_sentences_no_conct_dev.npy', allow_pickle=True)
# words_to_entitied_dev=np.load('words_to_entities_dev2.npy', allow_pickle=True)
# processed_sentences_train = np.load('processed_sentences_no_conct_train.npy', allow_pickle=True)
# words_to_entitied_train=np.load('words_to_entities_train2.npy', allow_pickle=True)
processed_sentences_test = np.load('processed_sentences_no_conct_test.npy', allow_pickle=True)
words_to_entitied_test=np.load('words_to_entities_test2.npy', allow_pickle=True)

In [3]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [4]:
def prepare_data(sentences, labels, tokenizer, max_length):
    input_ids = []
    attention_masks = []

    with tf.device('/CPU:0'):
        for sentence in tqdm(sentences, desc="Tokenizing sentences"):
            encoding = tokenizer(sentence,
                                 truncation=True,
                                 padding='max_length',
                                 max_length=max_length,
                                 return_tensors="tf")  # Use "tf" to return TensorFlow tensors

            input_ids.append(encoding["input_ids"])  # TensorFlow tensor
            attention_masks.append(encoding["attention_mask"])  # TensorFlow tensor
    
    # Convert the list of tensors into a single tensor (batch_size, sequence_length)
    input_ids = tf.concat(input_ids, axis=0)
    attention_masks = tf.concat(attention_masks, axis=0)

    # Convert labels to a TensorFlow tensor as well
    labels = tf.convert_to_tensor(labels)

    return input_ids, attention_masks, labels

In [5]:
entities=np.load('final_negated_entites2.npy', allow_pickle=True)
entities

array(['PIZZAORDER', 'DRINKORDER'], dtype='<U10')

In [6]:
entity_labels = [['B-'+entity, 'I-'+entity] for entity in entities]
entity_labels = [label for sublist in entity_labels for label in sublist]
# np.save('entity_labels2.npy', entity_labels)

In [7]:
entities_id = {e: i+1 for i, e in enumerate(entity_labels)}
entities_id['0']=0
entities_id['O']=0
entities_id

{'B-PIZZAORDER': 1,
 'I-PIZZAORDER': 2,
 'B-DRINKORDER': 3,
 'I-DRINKORDER': 4,
 '0': 0,
 'O': 0}

In [8]:
def padding_labels(labels, max_len):
    for i in range(len(labels)):
        if len(labels[i]) < max_len:
            labels[i] = labels[i] + ['0'] * (max_len - len(labels[i]))
        else:
            labels[i] = labels[i][:max_len]
    return labels

In [9]:
labels_train=[['0']+[entities_id[i] for i in tq]+['0'] for tq in words_to_entitied_train]
padded_labels_train = padding_labels(labels_train, 30)
padded_labels_train=np.array(padded_labels_train, dtype=np.int32)

In [10]:
input_ids_train, attention_masks_train, padded_labels_train = prepare_data(processed_sentences_train[:1000000], padded_labels_train[:1000000], tokenizer, 30)

I0000 00:00:1734265704.372842   75172 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4273 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
Tokenizing sentences: 100%|██████████| 1000000/1000000 [04:53<00:00, 3407.05it/s]


In [11]:
np.save('input_ids_train_position21.npy', input_ids_train)
np.save('attention_masks_train_position21.npy', attention_masks_train)
np.save('padded_labels_train_position21.npy', padded_labels_train)

In [10]:
input_ids_train, attention_masks_train, padded_labels_train = prepare_data(processed_sentences_train[1000000:2000000], padded_labels_train[1000000:2000000], tokenizer, 30)


I0000 00:00:1734266379.689275   76478 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4273 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
Tokenizing sentences: 100%|██████████| 1000000/1000000 [05:26<00:00, 3062.23it/s]


In [11]:
np.save('input_ids_train_position22.npy', input_ids_train)
np.save('attention_masks_train_position22.npy', attention_masks_train)
np.save('padded_labels_train_position22.npy', padded_labels_train)

In [10]:
input_ids_train, attention_masks_train, padded_labels_train = prepare_data(processed_sentences_train[2000000:], padded_labels_train[2000000:], tokenizer, 30)


I0000 00:00:1734267016.771709   77849 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4273 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
Tokenizing sentences: 100%|██████████| 456446/456446 [02:13<00:00, 3412.17it/s]


In [11]:
np.save('input_ids_train_position23.npy', input_ids_train)
np.save('attention_masks_train_position23.npy', attention_masks_train)
np.save('padded_labels_train_position23.npy', padded_labels_train)

In [11]:
labels_dev=[['0']+[entities_id[i] for i in tq]+['0'] for tq in words_to_entitied_dev]
padded_labels_dev = padding_labels(labels_dev, 30)
padded_labels_dev=np.array(padded_labels_dev, dtype=np.int32)

In [13]:
input_ids_dev, attention_masks_dev, padded_labels_dev = prepare_data(processed_sentences_dev, padded_labels_dev, tokenizer, 30)


I0000 00:00:1734265551.341278   74628 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4273 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
Tokenizing sentences: 100%|██████████| 348/348 [00:00<00:00, 3105.40it/s]


In [14]:
np.save('input_ids_dev_position2.npy', input_ids_dev)
np.save('attention_masks_dev_position2.npy', attention_masks_dev)
np.save('padded_labels_dev_position2.npy', padded_labels_dev)

In [2]:
input_ids_train_position=np.concat([np.load('input_ids_train_position21.npy'), np.load('input_ids_train_position22.npy'),np.load('input_ids_train_position23.npy')], axis=0)
attention_masks_train_position=np.concat([np.load('attention_masks_train_position21.npy'), np.load('attention_masks_train_position22.npy'),np.load('attention_masks_train_position23.npy')], axis=0)
padded_labels_train_position=np.concat([np.load('padded_labels_train_position21.npy'), np.load('padded_labels_train_position22.npy'),np.load('padded_labels_train_position23.npy')], axis=0)

np.save('input_ids_train_position222.npy', input_ids_train_position)
np.save('attention_masks_train_position222.npy', attention_masks_train_position)
np.save('padded_labels_train_position222.npy', padded_labels_train_position)

In [9]:
labels_test=[['0']+[entities_id[i] for i in tq]+['0'] for tq in words_to_entitied_test]
padded_labels_test = padding_labels(labels_test, 30)
padded_labels_test=np.array(padded_labels_test, dtype=np.int32)

In [10]:
input_ids_test, attention_masks_test, padded_labels_test = prepare_data(processed_sentences_test, padded_labels_test, tokenizer, 30)

I0000 00:00:1734280822.169608  125155 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1200 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
Tokenizing sentences: 100%|██████████| 1357/1357 [00:00<00:00, 3504.87it/s]


In [11]:
np.save('input_ids_test_position2.npy', input_ids_test)
np.save('attention_masks_test_position2.npy', attention_masks_test)
np.save('padded_labels_test_position2.npy', padded_labels_test)